In [ ]:
import imagehash
from PIL import Image
import os

def remove_duplicates(images_dir):
　　# 获取目录中的所有图像文件名
　　image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')]
　　# 创建一个字典用于存储哈希值和对应的图像文件名
　　hashes = {}
　　# 遍历所有图像文件
　　for image_file in image_files:
　　# 打开图像文件并计算哈希值
　　　　with Image.open(os.path.join(images_dir, image_file)) as img:
　　　　hash_value = str(imagehash.average_hash(img))
　　　　# 如果哈希值已经存在，说明是重复图像，将其删除
　　　　if hash_value in hashes:
　　　　　　os.remove(os.path.join(images_dir, image_file))
　　　　# 如果哈希值不存在，将其添加到字典中
　　　　else:
　　　　　　hashes[hash_value] = image_file

In [ ]:
import os
import shutil
import warnings
import cv2
import io
 
from PIL import Image
warnings.filterwarnings("error", category=UserWarning)
base_dir = "/data/chw/images"
i = 0
def is_read_successfully(file):
    try:
        imgFile = Image.open(file)
        return True
    except Exception:
        return False
 
for parent, dirs, files in os.walk(base_dir):
    for file in files:
        if not is_read_successfully(os.path.join(parent, file)):
            print(os.path.join(parent, file))
            os.remove(os.path.join(parent, file))
            i = i + 1
print(i)

In [ ]:
from sklearn import svm #引入svm包
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
#定义字典，将字符与数字对应起来
def Iris_label(s):
    it={b'Iris-setosa':0, b'Iris-versicolor':1, b'Iris-virginica':2}
    return it[s]
#读取数据，利用np.loadtxt()读取text中的数据
path='iris.data'              #将下载的原始数据放到项目文件夹，即可不用写路径
data= np.loadtxt(path, dtype=float, delimiter=',', converters={4:Iris_label}) #分隔符为‘,'
#确定输入和输出
x,y=np.split(data,(4,),axis=1)   #将data按前4列返回给x作为输入，最后1列给y作为标签值
#划分数据集和标签：利用sklearn中的train_test_split对原始数据集进行划分，本实验中训练集和测试集的比例为7：3。
train_data,test_data,train_label,test_label=train_test_split(x,y,random_state=1,train_size=0.7,test_size=0.3)
#创建svm分类器并进行训练：首先，利用sklearn中的SVC（）创建分类器对象，其中常用的参数有C（惩罚力度）、kernel（核函数）、gamma（核函数的参数设置）、decision_function_shape（因变量的形式），再利用fit()用训练数据拟合分类器模型。
'''C越大代表惩罚程度越大，越不能容忍有点集交错的问题，但有可能会过拟合（defaul C=1）；
kernel常规的有‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ ，默认的是rbf；
gamma是核函数为‘rbf’, ‘poly’ 和 ‘sigmoid’时的参数设置，其值越小，分类界面越连续，其值越大，分类界面越“散”，分类效果越好，但有可能会过拟合，默认的是特征个数的倒数；
decision_function_shape='ovr'时，为one v rest（一对多），即一个类别与其他类别进行划分，等于'ovo'时，为one v one（一对一），即将类别两两之间进行划分，用二分类的方法模拟多分类的结果。
'''
model=svm.SVC(C=2,kernel='rbf',gamma=10,decision_function_shape='ovo')
model.fit(train_data,train_label.ravel())    #ravel函数在降维时默认是行序优先
#利用classifier.score（）分别计算训练集和测试集的准确率。
train_score = model.score(train_data,train_label)
print("训练集：",train_score)
test_score = model.score(test_data,test_label)
print("测试集：",test_score)
#决策函数的查看（可省略）
#print('train_decision_function:\n',model.decision_function(train_data))#（90，3）
#pca降维
#直接使用sklearn中的PCA进行降维
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(x)
xx=pca.transform(x) # 降维后的结果
train_data_2,test_data_2,train_label_2,test_label_2=train_test_split(xx,y,random_state=1,train_size=0.7,test_size=0.3)#降维后的测试集划分
 
#训练集和测试集的预测结果
trainPredict = (model.predict(train_data).reshape(-1, 1))
testPredict = model.predict(test_data).reshape(-1, 1)
#将预测结果进行展示,首先画出预测点，再画出分类界面
#预测点的画法，可参考https://zhuanlan.zhihu.com/p/81006952
#画图例和点集
xx1_min,xx1_max=xx[:,0].min(),xx[:,0].max()   #x轴范围
xx2_min,xx2_max=xx[:,1].min(),xx[:,1].max()   #y轴范围
matplotlib.rcParams['font.sans-serif']=['SimHei']   #指定默认字体
cm_dark=matplotlib.colors.ListedColormap(['g','r','b'])  #设置点集颜色格式
cm_light=matplotlib.colors.ListedColormap(['#A0FFA0','#FFA0A0','#A0A0FF'])  #设置边界颜色
plt.xlabel('fea1',fontsize=13)        #x轴标注
plt.ylabel('fea2',fontsize=13)        #y轴标注
plt.xlim(xx1_min,xx1_max)                   #x轴范围
plt.ylim(xx2_min,xx2_max)                   #y轴范围
plt.title('SVM result')          #标题
plt.scatter(xx[:,0],xx[:,1],c=y[:,0],s=30,cmap=cm_dark)  #画出测试点
plt.scatter(test_data_2[:,0],test_data_2[:,1],c=testPredict[:,0],s=80,edgecolors='k',marker='+',zorder=2,cmap=cm_dark) #画出测试点，并将预测点圈出(注意这里是测试集的预测标签值) 
plt.show()

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 加载数据集
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建支持向量机分类器
clf = SVC(kernel='linear')

# 拟合模型
clf.fit(X_train, y_train)

# 预测
y_pred = clf.predict(X_test)

# 评估准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'准确率: {accuracy}')

In [ ]:
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage

%matplotlib inline

In [ ]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])

    classes = np.array(test_dataset["list_classes"][:])

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [ ]:
import cv2
dirpath = './image/1.jpg'
img_cv = cv2.imread(dirpath)
print('img_cv:',img_cv.shape)
print('img_cv:',type(img_cv))
print(img_cv)

In [ ]:
from sklearn import model_selection
# 将数据集拆分为训练集和测试集
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2, random_state = 1234)

In [ ]:
print("训练集长度：", len(X_train))
print("测试集长度：", len(X_test))

In [ ]:
import pandas as pd
data = pd.read_csv('C:\\path_to_your_dataset.csv', encoding='utf8')
# 特征变量
X = data[['特征1', '特征2', '特征3']]
# 目标变量
y = data['目标列']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
from sklearn.neighbors import KNeighborsClassifier
# 新建一个KNN模型，设置邻居个数
knn_model = KNeighborsClassifier(n_neighbors=3)
# 使用训练集训练KNN模型
knn_model.fit(X_train, y_train)
# 预测测试集
y_pred = knn_model.predict(X_test)
# 计算准确率
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'模型准确率: {accuracy:.2f}')

In [ ]:
from skimage import io, transform
import numpy as np
import os
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = io.imread(os.path.join(folder, filename))
        if img is not None:
            img = transform.resize(img, (100, 100))  # 调整图片大小
            images.append(img)
            labels.append(label)  # 根据文件名或路径设置标签
    return np.array(images), np.array(labels)
# 假设您的图片存储在'C:\\image_dataset\\train'和'C:\\image_dataset\\test'
X_train, y_train = load_images_from_folder('C:\\image_dataset\\train')
X_test, y_test = load_images_from_folder('C:\\image_dataset\\test')
# 将图片数据展平为一维数组
nsamples, nx, ny, nz = X_train.shape
X_train = X_train.reshape((nsamples,nx*ny*nz))

nsamples, nx, ny, nz = X_test.shape
X_test = X_test.reshape((nsamples,nx*ny*nz))
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(f'模型准确率: {accuracy_score(y_test, y_pred):.2f}')

In [ ]:
images.append(img)
labels.append(label)  # 根据文件名或路径设置标签
    return np.array(images), np.array(labels)